In [5]:
import sys
sys.path.append("/home/lau/postgap/lib/")

In [6]:
import postgap 
import postgap.Globals
import postgap.Integration
import sys
import numpy as np
import pandas as pd
import scipy
from postgap.Utils import *

In [7]:
import requests, sys
server = "http://rest.ensembl.org"

In [8]:
postgap.Globals.DATABASES_DIR = '/home/lau/postgap/databases'
postgap.Globals.SPECIES='human'

In [9]:
chromosome = '1'
pairs_f = '../github/data/GTEx_Analysis_v7_eQTL_EVB_linearData_hiC.csv'
rs_name = 'rs_id_dbSNP147_GRCh37p13'

In [94]:
data = pd.read_csv(pairs_f)
data = data[data['variant_id'].apply(lambda x: x.split('_')[0]==chromosome)]
print(data.shape)
data.head()

(396, 210)


,variant_id,gene_id,enhancer_id,promoter_id,enhancer_ATF2,enhancer_ATF3,enhancer_BATF,enhancer_BCL11A,enhancer_BCL3,enhancer_BCLAF1,...,promoter_ZNF143,promoter_ZNF384,promoter_ZZZ3,hiC_raw,hiC_kr,hiC_vc,hiC_sq,hiC_krExpected,hiC_vcExpected,hiC_sqExpected
3576,1_1239339_T_G_b37,ENSG00000127054,GM12878|chr1:1240136-1240200,GM12878|chr1:1309330-1312050,0.000000,0.0,0.0,0.0,0.0,0.0,...,492.157911,2.06497,0.0,23.0,24.927210,24.219449,28.722428,2.139792,1.709039,2.433242
3577,1_1241800_A_G_b37,ENSG00000127054,GM12878|chr1:1240136-1240200,GM12878|chr1:1309330-1312050,0.000000,0.0,0.0,0.0,0.0,0.0,...,492.157911,2.06497,0.0,23.0,24.927210,24.219449,28.722428,2.139792,1.709039,2.433242
3578,1_1242707_A_G_b37,ENSG00000127054,GM12878|chr1:1240136-1240200,GM12878|chr1:1309330-1312050,0.000000,0.0,0.0,0.0,0.0,0.0,...,492.157911,2.06497,0.0,23.0,24.927210,24.219449,28.722428,2.139792,1.709039,2.433242
3579,1_1242707_A_G_b37,ENSG00000127054,GM12878|chr1:1245169-1245389,GM12878|chr1:1309330-1312050,37.873228,0.0,0.0,0.0,0.0,0.0,...,492.157911,2.06497,0.0,24.0,34.794071,33.922299,34.723489,2.801549,2.588801,2.761328
3580,1_1297483_G_A_b37,ENSG00000242485,GM12878|chr1:1294400-1294888,GM12878|chr1:1406163-1407732,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.00000,0.0,6.0,12.878777,13.690376,11.029565,1.723805,1.792028,1.455436


In [17]:
rs = pd.read_csv('../../prototype/GTEx_Analysis_2016-01-15_v7_Chromosome1.lookup_table.txt.gz', sep='\t',\
                 header=-1)
rs.head()

,0,1,2,3,4,5,6
0,1,10478,1_10478_C_G_b37,C,G,1,rs528916756
1,1,10480,1_10480_C_A_b37,C,A,1,.
2,1,10482,1_10482_G_C_b37,G,C,1,.
3,1,10489,1_10489_C_T_b37,C,T,1,.
4,1,10497,1_10497_C_T_b37,C,T,1,.


In [38]:
rs.columns = ['chr','variant_pos','variant_id','ref','alt','num_alt_per_site', 'rs_id_dbSNP147_GRCh37p13']

In [42]:
rs.head()

,chr,variant_pos,variant_id,ref,alt,num_alt_per_site,rs_id_dbSNP147_GRCh37p13
0,1,10478,1_10478_C_G_b37,C,G,1,rs528916756
1,1,10480,1_10480_C_A_b37,C,A,1,.
2,1,10482,1_10482_G_C_b37,G,C,1,.
3,1,10489,1_10489_C_T_b37,C,T,1,.
4,1,10497,1_10497_C_T_b37,C,T,1,.


In [95]:
data = data.merge(rs, on=['variant_id'], how='inner')

In [44]:
rsIDs = data[rs_name].unique()

In [45]:
# extract SNP objects
snps = postgap.Ensembl_lookup.get_snp_locations(rsIDs)
print('Extracted {} SNPs.'.format(len(snps)))

Extracted 227 SNPs.


In [46]:
# get LD matrix
rsIDs, matrix = postgap.LD.get_pairwise_ld(snps, 'EUR')

In [47]:
len(rsIDs), len(set(rsIDs))

(209, 209)

In [96]:
data = data.loc[data[rs_name].isin(rsIDs),]

In [49]:
M = {}
for i, a in enumerate(rsIDs):
    if a not in M: M[a] = {}
    for j, b in enumerate(rsIDs):
        M[a][b] = matrix[i,j]

In [97]:
data['rs_name'] = data[rs_name] + '_' + data['promoter_id'] + data['enhancer_id'] 
data = data.drop_duplicates(subset='rs_name')

In [51]:
c = 0
n = 0
id_beta = {}
for i in data.index:
    gene_stbl_id = data.loc[i,]['gene_id']
    variant_name = data.loc[i,][rs_name]
    ext = "/eqtl/id/homo_sapiens/"+gene_stbl_id+"?statistic=beta;variant_name="+variant_name+";tissue=Cells_EBV-transformed_lymphocytes" 
    r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})
    if not r.ok:
        try:
            r.raise_for_status()
        except:
            n += 1
    try:
        decoded = r.json()
    except:
        n += 1
    if "error" not in repr(decoded) and decoded != []: 
        if (gene_stbl_id, variant_name) not in id_beta:
            id_beta[(gene_stbl_id, variant_name)] = decoded[0]['value']
        else:
            if decoded[0]['value'] != id_beta[(gene_stbl_id, variant_name)]:
                print('ERROR!')
        c += 1
print(c, n)

ERROR!
ERROR!
(355, 26)


In [52]:
ib = {}
keys = tuple(id_beta.keys())
ib['gene_id'], ib[rs_name] = zip(*keys)
ib['beta'] = tuple(id_beta[k] for k in keys)

In [98]:
data = data.merge(pd.DataFrame(ib), right_on=['gene_id', rs_name], left_on=['gene_id', rs_name])
data.shape

(355, 218)

In [54]:
E = []
for a in data[rs_name]:
    e = []
    for b in data[rs_name]:
        e.append(M[a][b])
    E.append(e)
    
E = np.array(E)

In [56]:
evb = pd.read_csv('/mnt/c/Users/laura/Documents/EBI_internship/TFG/github/data/GTEx_Analysis_v7_eQTL/Cells_EBV-transformed_lymphocytes.v7.signif_variant_gene_pairs.txt.gz', sep='\t')
evb.head()

,variant_id,gene_id,tss_distance,ma_samples,ma_count,maf,pval_nominal,slope,slope_se,pval_nominal_threshold,min_pval_nominal,pval_beta
0,1_1511837_C_G_b37,ENSG00000225972.1,947395,7,8,0.034188,1.104660e-05,-1.152090,0.248181,0.000018,1.104660e-05,0.015085
1,1_710088_C_A_b37,ENSG00000224956.5,46561,4,5,0.021368,2.210340e-07,-1.401700,0.250950,0.000014,6.974990e-09,0.000016
2,1_710195_A_G_b37,ENSG00000224956.5,46668,5,6,0.025641,1.608950e-08,-1.447660,0.234342,0.000014,6.974990e-09,0.000016
3,1_712515_T_TC_b37,ENSG00000224956.5,48988,15,17,0.080952,1.213410e-07,-0.927573,0.162069,0.000014,6.974990e-09,0.000016
4,1_713092_G_A_b37,ENSG00000224956.5,49565,4,5,0.021368,2.210340e-07,-1.401700,0.250950,0.000014,6.974990e-09,0.000016


In [58]:
evb['gene_id'] = evb.gene_id.apply(lambda x: x.split('.')[0])

In [99]:
data = data.merge(evb, on=['variant_id', 'gene_id'], how='inner')

In [80]:
data_z = data[['rs_name','chr','variant_pos','ref', 'alt', 'maf', 'beta', 'slope_se']]
data_z.columns = ['rsid', 'chromosome','position','allele1', 'allele2', 'maf', 'beta', 'se']

In [81]:
data_z.shape, E.shape

((355, 8), (355, 355))

In [82]:
data_z.to_csv('../data/chromosome'+chromosome+'_to_run_finemap.z', index=False, sep=' ')

In [83]:
np.savetxt('../data/chromosome'+chromosome+'_to_run_finemap.ld', E, delimiter=' ')

In [84]:
# write the master:
headers = 'z;ld;snp;config;cred;log;n_samples\n'
names   = 'data/chromosome'+chromosome+'_to_run_finemap.z;data/chromosome'+chromosome+'_to_run_finemap.ld;\
data/chromosome'+chromosome+'_to_run_finemap.snp;data/chromosome'+chromosome+'_to_run_finemap.config;data/chromosome'+chromosome+'_to_run_finemap.cred;data/chromosome'+chromosome+'_to_run_finemap.log;'+str(data_z.shape[0])

In [85]:
with open('../data/master', 'w') as f:
    f.write(headers+names)

### To run fineMap.
You need to download the Github repo and follow the instructions there. 
Then add the path of the program to your path. 
Also, note that the verision is v1.3.1!                                                     

In [110]:
snp = pd.read_table('../data/chromosome1_to_run_finemap.snp', sep=' ', index_col=0)
snp.head()

/home/lau/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.


,rsid,chromosome,position,allele1,allele2,maf,beta,se,z,prob,log10bf,group,corr_group,prob_group,log10bf_group,mean,sd,mean_incl,sd_incl
index,,,,,,,,,,,,,,,,,,,
299,rs1242331_GM12878|chr1:53017504-53019678GM1287...,1,53392908,A,G,0.276316,0.611292,0.075897,8.05426,1.00000,12.35050,299,1,1,12.3505,0.646919,0.028543,0.646919,0.028543
211,rs3768543_GM12878|chr1:226068902-226071144GM12...,1,226253926,T,C,0.256410,0.898322,0.097062,9.25512,1.00000,12.35050,211,1,1,12.3505,0.832843,0.029309,0.832843,0.029309
260,rs12039069_GM12878|chr1:33430151-33430865GM128...,1,33854013,C,T,0.295652,0.735142,0.086737,8.47556,1.00000,12.35050,260,1,1,12.3505,0.701754,0.031670,0.701754,0.031670
197,rs11580071_GM12878|chr1:161037430-161040320GM1...,1,160951832,G,A,0.094017,1.014930,0.143916,7.05226,1.00000,9.59909,197,1,1,12.3505,0.815989,0.044897,0.815989,0.044897
90,rs2039590_GM12878|chr1:150737600-150738647GM12...,1,150813315,G,A,0.465812,-0.484168,0.061026,-7.93379,0.61039,2.54558,90,1,1,12.3505,-0.345363,0.276615,-0.565807,0.025042


In [111]:
data = data.merge(snp, right_on='rsid', left_on='rs_name')

In [103]:
data.to_csv('../github/data/GTEx_Analysis_v7_eQTL_EVB_linearData_hiC_z.csv', index=False)